### Mainly inspired by this [github gist](https://gist.github.com/raveenb/c0001484a79214c21785227d0688f57f)

- the gist implements `ngrok` as well

### [Example of uploading to S3 bucket](https://github.com/gauravgola96/FastAPI-Example/blob/master/api.py)

-  interesant de folosit in viitor

### TODOS

- [x] find way to delete image after prediction is made
- [ ] try to put on `heroku`
- [ ] guard: implement image upload cannot be bigger than 12MB


## Imports

In [1]:
from fastapi import FastAPI, File, UploadFile, Query
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
import uvicorn
import nest_asyncio
from starlette.responses import StreamingResponse
# fast ai imports
from fastai.vision.all import *
from fastai import *
# cloudinary imports
from cloudinary import CloudinaryImage
import cloudinary.uploader
import cloudinary.api
# misc imports
import shutil
import io

## Initialize `fast api`

In [2]:
app = FastAPI()

## Load Model

In [3]:
path = Path()
learner = load_learner(path/'export.pkl', cpu=True)
image_store_path = path/"images/"

### <font color="red">To use `Cloudinary` urls as a start for an image we need to [configure cloudinary](https://cloudinary.com/documentation/django_integration#configuration)</font>

In [5]:
#class CloudinaryLink(BaseModel):
    #img_url: str

In [6]:
#@app.post("/cloudinary/")
#async def create_pred_from_url(img_url: CloudinaryLink):
    #img = tensor(Image.open(img_url))
    #pred, pred_idx, probs = learner.predict(img)
    #return jsonable_encoder({
        #f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    #})

## Routes

In [4]:
@app.get("/")
async def root():
    return {"message": "Fast API - Upload image and classify"}

### <font color="red">Predict Route</font>

In [7]:
@app.post("/predict/")
async def create_upload_file(file: UploadFile = File(...)):
    temp_file = file.file
    # save file to disk
    with open(f"{image_store_path}/{file.filename}", "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)
        #os.remove(file.filename)
    # get file and prediction
    img = tensor(Image.open(image_store_path/file.filename))
    pred, pred_idx, probs = learner.predict(img)
    # remove image from disk after predicition
    os.remove(image_store_path/file.filename)
    
    return jsonable_encoder({
        f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    })

## Start the server 

In [8]:
nest_asyncio.apply()

In [ ]:
if __name__ == '__main__':    
    uvicorn.run(app, host='127.0.0.1', port=9000)

INFO:     Started server process [5176]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:9000 (Press CTRL+C to quit)


INFO:     127.0.0.1:47212 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     127.0.0.1:47220 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     127.0.0.1:47264 - "POST /predict/ HTTP/1.1" 200 OK
